# Multi-Agent with OpenAI

In [1]:
import os
from databricks.connect import DatabricksSession

# if running on a local IDE
spark = DatabricksSession.builder.remote(serverless=True).getOrCreate()

In [2]:
import os
import mlflow

# TODO make sure you update the config file before this

configs = mlflow.models.ModelConfig(development_config="./config.yaml")
databricks_configs = configs.get("databricks")
agent_configs = configs.get("agent")

CATALOG = databricks_configs["catalog"]
SCHEMA = databricks_configs["schema"]
UC_MODEL = databricks_configs["model"]
WORKSPACE_URL = databricks_configs["workspace_url"]
SQL_WAREHOUSE_ID = databricks_configs["sql_warehouse_id"]
MLFLOW_EXPERIMENT_ID = databricks_configs["mlflow_experiment_id"]
AGENT_NAME = agent_configs["name"]

In [3]:
import mlflow

mlflow.set_registry_uri("databricks-uc")
mlflow.set_tracking_uri("databricks")

try:
    experiment = mlflow.get_experiment(experiment_id=MLFLOW_EXPERIMENT_ID)
    mlflow.set_experiment(experiment_id=MLFLOW_EXPERIMENT_ID)
    print(f"Set to existing experiment:\n")
    print(experiment)
except mlflow.exceptions.RestException as e:
    if "does not exist" in str(e):
        print(f"Experiment not found. Must create one first.")
    else:
        raise e

Set to existing experiment:

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2254899697754052', creation_time=1757315578511, experiment_id='2254899697754052', last_update_time=1757315578511, lifecycle_stage='active', name='/Users/david.huang@databricks.com/client-demos/agent/oai-response-agent/oai_gdr_agent_exp', tags={'mlflow.experiment.sourceName': '/Users/david.huang@databricks.com/client-demos/agent/oai-response-agent/oai_gdr_agent_exp',
 'mlflow.experimentKind': 'genai_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'david.huang@databricks.com',
 'mlflow.ownerId': '2635198831033931'}>


## Load & test agent

Make sure you go to the MLflow experiment to look at trace data as you develop & debug the agent.

In [4]:
%run ./agent.py

In [5]:
sample_questions = [
    # "Compare and contrast the annual net income growth in the past 10 years between BAC and AXP",
    "What's the debt-to-asset ratio for American Express in the past 10 years, compare to that of Bank of America?",
]

input_example = {
    "input": [
        {
            "role": "user",
            "content": sample_questions[0],
        }
    ]
}

In [6]:
result = AGENT.predict(input_example)

print(result.model_dump()["output"][-1]["content"][0]["text"])

## Debt-to-Asset Ratio Analysis: American Express vs. Bank of America (2013-2022)

Based on the data analysis, here are the key findings comparing the debt-to-asset ratios of American Express and Bank of America over the past 10 years:

### Key Insights:

1. **Overall Comparison**: 
   - American Express has consistently maintained a significantly higher debt-to-asset ratio than Bank of America throughout the entire period.
   - The average debt-to-asset ratio for American Express was 0.3060 compared to Bank of America's 0.1129.
   - The average difference between the two companies was 0.1931.

2. **Trends Over Time**:
   - Both companies have been reducing their debt-to-asset ratios over the 10-year period.
   - American Express showed a more substantial reduction of 0.2005 (50.95%), decreasing from 0.3935 in 2013 to 0.1930 in 2022.
   - Bank of America reduced its ratio by 0.0413 (29.37%), from 0.1406 in 2013 to 0.0993 in 2022.

3. **Year-by-Year Comparison**:
   - In 2013, American 

In [ ]:
for event in AGENT.predict_stream(input_example):
    print(event, "-----------\n")